# Lab 12: Authentication and Security

## Learning Objectives
- Implement JWT-based authentication
- Configure password hashing with bcrypt
- Create user authentication functions
- Implement role-based access control
- Define security dependencies

**Duration:** 45 minutes  
**Prerequisites:** Completion of API Setup and Configuration

## Step 1: Authentication System Setup

This cell implements JWT-based authentication with password hashing and token management.

In [ ]:
# Cell 5: JWT-based authentication system
from fastapi import HTTPException, Depends, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta
import secrets

# Security configuration
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
security = HTTPBearer()

class AuthenticationManager:
    """Handles JWT authentication and authorization"""
    
    def __init__(self, secret_key: str, algorithm: str = "HS256"):
        self.secret_key = secret_key
        self.algorithm = algorithm
    
    def create_access_token(self, data: Dict[str, Any], expires_delta: Optional[timedelta] = None):
        """Create JWT access token"""
        to_encode = data.copy()
        
        if expires_delta:
            expire = datetime.utcnow() + expires_delta
        else:
            expire = datetime.utcnow() + timedelta(minutes=CONFIG["access_token_expire_minutes"])
        
        to_encode.update({"exp": expire})
        encoded_jwt = jwt.encode(to_encode, self.secret_key, algorithm=self.algorithm)
        
        return {
            "access_token": encoded_jwt,
            "token_type": "bearer",
            "expires_in": int(expires_delta.total_seconds()) if expires_delta else CONFIG["access_token_expire_minutes"] * 60,
            "user_id": data.get("sub"),
            "role": data.get("role")
        }
    
    def verify_token(self, token: str) -> Dict[str, Any]:
        """Verify and decode JWT token"""
        try:
            payload = jwt.decode(token, self.secret_key, algorithms=[self.algorithm])
            return payload
        except JWTError:
            raise HTTPException(
                status_code=status.HTTP_401_UNAUTHORIZED,
                detail="Could not validate credentials",
                headers={"WWW-Authenticate": "Bearer"},
            )
    
    def hash_password(self, password: str) -> str:
        """Hash password using bcrypt"""
        return pwd_context.hash(password)
    
    def verify_password(self, plain_password: str, hashed_password: str) -> bool:
        """Verify password against hash"""
        return pwd_context.verify(plain_password, hashed_password)

# Initialize authentication manager
auth_manager = AuthenticationManager(CONFIG["secret_key"], CONFIG["algorithm"])

def get_current_user(credentials: HTTPAuthorizationCredentials = Depends(security)) -> Dict[str, Any]:
    """Dependency to get current authenticated user"""
    return auth_manager.verify_token(credentials.credentials)

def require_role(required_role: UserRole):
    """Dependency factory for role-based access control"""
    def role_checker(current_user: Dict[str, Any] = Depends(get_current_user)):
        user_role = current_user.get("role")
        if user_role != required_role.value and user_role != "admin":
            raise HTTPException(
                status_code=status.HTTP_403_FORBIDDEN,
                detail=f"Access denied. Required role: {required_role.value}"
            )
        return current_user
    return role_checker

# Create demo users in database
def create_demo_users():
    """Create demo users for API testing"""
    demo_users = [
        {
            "user_id": "user_001",
            "username": "admin",
            "email": "admin@insurance.com",
            "full_name": "System Administrator",
            "password_hash": auth_manager.hash_password("admin123"),
            "role": "admin",
            "is_active": True
        },
        {
            "user_id": "user_002", 
            "username": "agent1",
            "email": "agent1@insurance.com",
            "full_name": "John Agent",
            "password_hash": auth_manager.hash_password("agent123"),
            "role": "agent",
            "is_active": True
        },
        {
            "user_id": "user_003",
            "username": "customer1",
            "email": "customer1@email.com", 
            "full_name": "Jane Customer",
            "password_hash": auth_manager.hash_password("customer123"),
            "role": "customer",
            "is_active": True
        }
    ]
    
    for user_data in demo_users:
        create_user_query = """
        MERGE (u:User {user_id: $user_id})
        SET u += {
            username: $username,
            email: $email,
            full_name: $full_name,
            password_hash: $password_hash,
            role: $role,
            is_active: $is_active,
            created_date: datetime()
        }
        RETURN u.username as username
        """
        
        result = connection_manager.execute_write_query(create_user_query, user_data)
        if result:
            print(f"✓ Demo user created: {user_data['username']}")

create_demo_users()
print("✓ Authentication system configured")
print("✓ Demo users created (admin/admin123, agent1/agent123, customer1/customer123)")

## Step 2: Authentication Endpoints

Define the login, profile, and logout endpoints for user authentication.

In [ ]:
# Cell 6: Authentication API endpoints
@app.post("/auth/login", response_model=Token, tags=["Authentication"])
async def login(login_data: UserLogin):
    """Authenticate user and return JWT token"""
    
    # Query user from database
    query = """
    MATCH (u:User {username: $username, is_active: true})
    RETURN u.user_id as user_id, u.username as username, u.email as email,
           u.full_name as full_name, u.password_hash as password_hash, 
           u.role as role, u.created_date as created_date
    """
    
    result = connection_manager.execute_query(query, {"username": login_data.username})
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid username or password"
        )
    
    user = result[0]
    
    # Verify password
    if not auth_manager.verify_password(login_data.password, user["password_hash"]):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid username or password"
        )
    
    # Create access token
    token_data = {
        "sub": user["user_id"],
        "username": user["username"],
        "role": user["role"],
        "email": user["email"]
    }
    
    token = auth_manager.create_access_token(token_data)
    
    return Token(**token)

@app.get("/auth/profile", response_model=UserProfile, tags=["Authentication"])
async def get_profile(current_user: Dict[str, Any] = Depends(get_current_user)):
    """Get current user profile"""
    
    query = """
    MATCH (u:User {user_id: $user_id})
    RETURN u.user_id as user_id, u.username as username, u.email as email,
           u.full_name as full_name, u.role as role, u.is_active as is_active,
           u.created_date as created_date
    """
    
    result = connection_manager.execute_query(query, {"user_id": current_user["sub"]})
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User profile not found"
        )
    
    user = result[0]
    return UserProfile(**user)

@app.post("/auth/logout", tags=["Authentication"])
async def logout(current_user: Dict[str, Any] = Depends(get_current_user)):
    """Logout user (client should discard token)"""
    return APIResponse(message=f"User {current_user['username']} logged out successfully")

print("✓ Authentication endpoints configured")